In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os,errno
import sys
import pandas as pd
import scipy.stats as st
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
#@author: Michael Schramm on GitHub
#This function is derived from code originally posted by Sat Kumar Tomer
#(satkumartomer@gmail.com)
#See also: http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm

from scipy.stats import norm
import scipy.stats as st
def mk_test(x, alpha=0.05):
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [ ]:
names=[dir+'ls_extreme_rain_taiwan_90_70.nc',dir+'ls_extreme_rain_taiwan_90_80.nc',
       dir+'ls_extreme_rain_taiwan_90_90.nc',dir+'ls_extreme_rain_taiwan_95_70.nc',
      dir+'ls_extreme_rain_taiwan_95_80.nc',dir+'ls_extreme_rain_taiwan_95_90.nc',
      dir+'ls_extreme_rain_taiwan_99_70.nc',dir+'ls_extreme_rain_taiwan_99_80.nc',
      dir+'ls_extreme_rain_taiwan_99_90.nc']
#names=[dir+'ls_extreme_rain_taiwan_99_90.nc']
for name in names:
    print(name)
    dp=xr.open_dataset(dir+'station_numbers.nc')
    stations=dp.stations.values.tolist()

    yrs_taiwan=np.arange(1979,2020,1)
    ds=xr.open_dataset(name)
    
    dates_unique=ds['large_scale_extreme_days'].values
    dates_unique_pd=pd.DatetimeIndex(dates_unique)
    years=[x.year for x in dates_unique_pd]
    rain_all=ds['large_scale_extreme_rain_all']
    
    mean_obs_list=[]
    for i in range(len(dates_unique)):
        obs=rain_all[i,:,:]
        mean_obs=obs.mean(dim=('lat','lon'),skipna=True).values#mean over whole island
        mean_obs_list.append(mean_obs)
    
    zipped_years=list(zip(years,mean_obs_list))
    values_list=[]
    for i in range(len(yrs_taiwan)):
        select_year=[x for x in zipped_years if x[0]==yrs_taiwan[0]+i]
        obs_year=[x[1] for x in select_year]
        if len(obs_year)>1:
            total_year=sum(obs_year)*30./91.
        if len(obs_year)==1:
            total_year=obs_year[0]*30./91.
        if len(obs_year)==0:
            total_year=0
        values_list.append(total_year)
        
    r1=st.linregress(np.arange(len(yrs_taiwan)),values_list)
    t1=st.theilslopes(values_list,yrs_taiwan,alpha=0.95)
    mean_value=float(sum(values_list))/float(len(values_list))
    slope_percent_1a=t1[0]/mean_value
    slope_percent_1b=r1[0]/mean_value
    mk_result=mk_test(values_list)
    print(len(dates_unique))
    print(slope_percent_1a)
    print(slope_percent_1b)
    print(mk_result)
    print(r1)
    
    if 1==1:
        fig=plt.figure(figsize=(8,8))
        ax=plt.subplot(1,1,1)
        ax.plot(yrs_taiwan,values_list,color='r',linewidth=1.0,linestyle='solid',label='Fall EP day sum')
        ax.plot(yrs_taiwan,[r1[0]*x+r1[1] for x in np.arange(len(yrs_taiwan))],color='k',linestyle='--',linewidth=1.5,label='y='+str(round(r1[0],2))+'x+'+str(round(r1[1],2))+', p='+str(round(r1[3],2)))
        ax.tick_params(labelsize=14)
        ax.set_xlabel('Year',fontsize=14)
        ax.set_ylabel('Monthly spatially-averaged precipitation (mm)',fontsize=14)
        ax.set_title('Taiwan Seasonal EP day precipitation',fontsize=18)
        ax.set_ylim(0,200)
        plt.legend(loc='upper left',fontsize=12)
        plt.show()